In [ ]:
def distance(model, x) -> float:
    w = model.coef_.flatten()  # Extract weight vector
    b = model.intercept_[0]    # Extract bias term
    
    # Compute perpendicular distance
    distance = np.abs(np.dot(w, x) + b) / np.linalg.norm(w)
    
    return distance

In [ ]:
predictions = []
n = x.shape[0]
for i in range(n):
    predictions.append(model.predict(x[i].reshape(1,-1))[0]) # This is the initial model h*

def get_fairness_subset(X, predictions, indices):
    num_0 = 0 
    num_1 = 0 
    sum_0 = 0 
    sum_1 = 0 

    for i in indices:
        if(X[i,2]): 
            num_1 += 1 
            sum_1 += predictions[i] == 1 

        else:
            num_0 += 1 
            sum_0 += predictions[i] == 1 
    
    if(num_0 > 0 and num_1 > 0): return np.abs(sum_1/num_1 - sum_0/num_0)
    elif(num_0 == 0): return np.abs(sum_1/num_1)
    return np.abs(sum_0/num_0)

def fairness_set(subset):
    num_0 = 0 
    sum_0 = 0 
    num_1 = 0 
    sum_1 = 0

    num_1 = len(subset[0]) + len(subset[1]) 
    num_0 = len(subset[2]) + len(subset[3]) 
    sum_1 = len(subset[1])
    sum_0 = len(subset[3])
    
    if(num_0 > 0 and num_1 > 0): return np.abs(sum_1/num_1 - sum_0/num_0)
    elif(num_0 == 0): return np.abs(sum_1/num_1)
    return np.abs(sum_0/num_0)

In [ ]:
u_S = get_fairness_subset(x, predictions, range(n))
best_subset = [set(), set(), set(), set()]

X_female_one = set()
X_female_zero = set()
X_male_one = set()
X_male_zero = set()

for i in range(n):
    dist = distance(model, x[i]) 
    if(x[i,2]):
        if(predictions[i]): X_female_one.add((dist,i))
        else: X_female_zero.add((dist,i))
    else:
        if(predictions[i]): X_male_one.add((dist,i))
        else: X_male_zero.add((dist,i))

group_1_0 = len(X_female_one) + len(X_male_zero) 
group_0_1 = len(X_female_zero) + len(X_male_one) 


done = 0 
if(group_1_0 > group_0_1):
    # SELECTSAMPLES-G1
    while(done < budget):
        if(len(X_female_one) > 0):
            new_el = list(X_female_one)[0]
            X_female_one.remove(new_el)
            best_subset[1].add(new_el) 
            done+=1

        elif(len(X_female_zero) > 0):
            new_el = list(X_female_zero)[0]
            X_female_zero.remove(new_el)
            best_subset[0].add(new_el) 
            done+=1

        if(len(X_male_zero) > 0):
            new_el = list(X_male_zero)[0]
            X_male_zero.remove(new_el)
            best_subset[2].add(new_el) 
            done+=1

        elif(len(X_male_one) > 0):
            new_el = list(X_male_one)[0]
            X_male_one.remove(new_el)
            best_subset[3].add(new_el) 
            done+=1

else:
    while(done < budget):
        # SELECTSAMPLES-G0
        if(len(X_female_zero) > 0):
            new_el = list(X_female_zero)[0]
            X_female_zero.remove(new_el)
            best_subset[0].add(new_el) 
            done+=1

        elif(len(X_female_one) > 0):
            new_el = list(X_female_one)[0]
            X_female_one.remove(new_el)
            best_subset[1].add(new_el) 
            done+=1

        if(len(X_male_one) > 0):
            new_el = list(X_male_one)[0]
            X_male_one.remove(new_el)
            best_subset[3].add(new_el) 
            done+=1

        elif(len(X_male_zero) > 0):
            new_el = list(X_male_zero)[0]
            X_male_zero.remove(new_el)
            best_subset[2].add(new_el) 
            done+=1
   

u_h = fairness_set(best_subset)

while np.abs(u_h - u_S > alpha):

    if(group_1_0 > group_0_1):
        # ADJUSTSAMPLES-G1
        if(len(X_female_zero) > 0):
            new_el = list(X_female_zero)[0]
            X_female_zero.remove(new_el)
            best_subset[0].add(new_el) 

        if(len(best_subset[1]) > 0):
            remove_el = list(best_subset[1])[-1]
            best_subset[1].remove(remove_el)
            X_female_one.add(remove_el)

        if(len(X_male_one) > 0):
            new_el = list(X_male_one)[0]
            X_male_one.remove(new_el)
            best_subset[3].add(new_el) 

        if(len(best_subset[2]) > 0):
            remove_el = list(best_subset[2])[-1]
            best_subset[2].remove(remove_el)
            X_male_zero.add(remove_el)

    else:
        # ADJUSTSAMPLES-G0
        if(len(X_female_one) > 0):
            new_el = list(X_female_one)[0]
            X_female_one.remove(new_el)
            best_subset[1].add(new_el) 

        if(len(best_subset[0]) > 0):
            remove_el = list(best_subset[0])[-1]
            best_subset[0].remove(remove_el)
            X_female_zero.add(remove_el)

        if(len(X_male_zero) > 0):
            new_el = list(X_male_zero)[0]
            X_male_zero.remove(new_el)
            best_subset[2].add(new_el) 

        if(len(best_subset[3]) > 0):
            remove_el = list(best_subset[3])[-1]
            best_subset[3].remove(remove_el)
            X_male_one.add(remove_el)

    u_h = fairness_set(best_subset)
            
best_subset_open = []
for cat in best_subset:
    for s in cat:
        best_subset_open.append(s[1])

u_h1 = mu(find_h1(model, best_subset_open, x),x)
u_h2 = mu(find_h2(model, best_subset_open, x),x)
print(max(np.max([np.abs(u_h1), np.abs(u_h2)]) - u_S,0))